In [1]:
import pandas as pd
import os

path='data'

ref_file_path=os.path.join(path,'refrigerator_clustered.dat')
ref_dataset=pd.read_csv(ref_file_path, sep=",", usecols=[0,2],header=None).values
print(ref_dataset.shape)
#from random import shuffle
#shuffle(ref_dataset)

FileNotFoundError: [Errno 2] File b'data\\refrigerator_clustered.dat' does not exist: b'data\\refrigerator_clustered.dat'

In [ ]:
print(ref_dataset.shape)
ref_dataset_new=[]

ref_bins=4
ref_window_size=2401

bin_max_count=[0 for i in range(ref_bins)]

In [ ]:
print(ref_dataset[:3])

import collections
print('Ref bins:',collections.Counter(ref_dataset[:,1]))

In [ ]:
import collections
bin_dict=collections.Counter(ref_dataset[:,1])
key_min = min(bin_dict.keys(), key=(lambda k: bin_dict[k]))
ref_min_bin=bin_dict[key_min]
print(ref_min_bin)
for i in range(ref_dataset.shape[0]-ref_window_size-1):
    if(ref_dataset[i,1]==0):
        bin_max_count[0]=bin_max_count[0]+1
        if(bin_max_count[0]<ref_min_bin-1):
            ref_dataset_new.append([ref_dataset[i:i+ref_window_size,0],ref_dataset[i,1]])
    elif(ref_dataset[i,1]==1):
        bin_max_count[1]=bin_max_count[1]+1
        if(bin_max_count[1]<ref_min_bin-1):
            ref_dataset_new.append([ref_dataset[i:i+ref_window_size,0],ref_dataset[i,1]])
    elif(ref_dataset[i,1]==2):
        bin_max_count[2]=bin_max_count[2]+1
        if(bin_max_count[2]<ref_min_bin-1):
            ref_dataset_new.append([ref_dataset[i:i+ref_window_size,0],ref_dataset[i,1]])
    elif(ref_dataset[i,1]==3):
        bin_max_count[3]=bin_max_count[3]+1
        if(bin_max_count[3]<ref_min_bin-1):
            ref_dataset_new.append([ref_dataset[i:i+ref_window_size,0],ref_dataset[i,1]])

In [ ]:
import numpy as np

dish_dataset_new=np.array(ref_dataset_new)

data_dish=dish_dataset_new[:,0]
target_dish=dish_dataset_new[:,1]

from keras.utils import np_utils
target_dish = np_utils.to_categorical(target_dish)

In [ ]:
data_dish=np.stack(data_dish)
data_dish=data_dish/500.0
#data_dish = normalize(data_dish, axis=1)
print(data_dish.shape,data_dish[:5])
data_dish=data_dish.reshape(data_dish.shape[0],data_dish.shape[1],1)
#data_dish=data_dish.reshape(data_dish.shape[0],data_dish.shape[1])

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data_dish,target_dish,test_size=0.1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import normalize
from keras.layers import BatchNormalization

def load_dish_cnn():
    
    model=Sequential()

    model.add(Conv1D(filters=8,kernel_size=5,input_shape=train_data.shape[1:]))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(4,activation='sigmoid'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 2
    momentum = 0.8
    opt= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
import keras

mc = keras.callbacks.ModelCheckpoint('wights_with_artificial_data/NEW/weights_NEW{epoch:08d}.h5', save_weights_only=True)
model=load_dish_cnn()
#model.load_weights('wights_with_artificial_data/weights_NEW00000266.h5')
history=model.fit(train_data,train_target,epochs=10000,callbacks=[mc],)